## Introduction to `robotoc` 
# 2: Robot Models

## The goal of this chapter
Here, we learn how to create robot models (`robotoc::Robot`) from URDF packages.  
Note that, `robotoc::Robot` is just a wrapper of [`Pinocchio`](https://github.com/stack-of-tasks/pinocchio)'s API.  
So, if you want to dive in deeper, please check [`Pinocchio`'s documentation](https://stack-of-tasks.github.io/pinocchio/).

## What is URDF?
URDF (universal robot description format) describes the robot.  
We need very little knowledge on URDF as long as we use off-the-shelf URDF files with `robotoc`.  
So don't worry even if you do not know well about URDF!  
You can see off-the-shelf URDF packages in this repository's `urdf` folder.    
As we can see there, a URDF package is composed of urdf file (`~.urdf`) and mesh files (e.g., `~.stl`, `~.dae`, or `~.obj`)   
The urdf file describes the tree structure of the multi-link robot and mesh files contain visual and collision information.   
If you'd like to know more, please see, e.g., the ROS tutorials http://wiki.ros.org/urdf/Tutorials.  

## Simplest example: a robot manipulator `iiwa14`

First, we construct `robotoc.Robot` (`robotoc::Robot` in C++) with a simple example.  
Note that C++'s `robotoc::Robot` has more rich methods than the following Python counterpart.

In [ ]:
import robotoc
path_to_urdf = "urdf/iiwa_description/urdf/iiwa14.urdf"
iiwa14 = robotoc.Robot(path_to_urdf)

We can see the information of the robot as 

In [ ]:
print(iiwa14)

The printed information is as follows:
- `dimq`: the dimension of the configuration
- `dimv`: the dimension of the generalized velocity (not always the same as `dimq`)
- `dimu`: the dimension of the input torque
- `dim_passive`: the dimension of the passive joints including the floating base
- `frames`: frames of the robot. you can use to design the cost function and constraints on the end-effector.
- `joints`: each joint info.
- `effort limit`: limits of joint torques
- `velocity limit`: limits of joint velocities
- `lower position limit`, `upper position limit`: limits of joint positions

You can individually get some of them as

In [ ]:
iiwa14.dimq()

In [ ]:
iiwa14.dimv()

In [ ]:
iiwa14.dimu()

In [ ]:
iiwa14.dim_passive()

In [ ]:
iiwa14.joint_effort_limit()

In [ ]:
iiwa14.joint_velocity_limit()

In [ ]:
iiwa14.lower_joint_position_limit()

In [ ]:
iiwa14.upper_joint_position_limit()

Next, let's visualize them with some random configuration

In [ ]:
steps = 10
q = [iiwa14.generate_feasible_configuration() for i in range(steps)]
dt = 1.0

viewer = robotoc.utils.TrajectoryViewer(path_to_urdf=path_to_urdf, viewer_type='meshcat')
viewer.display(dt, q)

You can also call simple forward kinematics with the robot object.

In [ ]:
end_effector_frame = 'iiwa_link_ee_kuka'
for e in q:
    iiwa14.forward_kinematics(e)
    print('q: ', e)
    print('frame position: ', iiwa14.frame_position(end_effector_frame))
    print('frame rotation: ', iiwa14.frame_rotation(end_effector_frame))
    print('frame placement (SE3): ', iiwa14.frame_placement(end_effector_frame))

## Quadrupedal robot `ANYmal`

Second, we try a quadruped robot example.  
The quadrupedal robot typically involves **four point contacts**.  
So we specify it in the constructor as the names of the contact frames or contact frame indices.  
Here, we do it via names.

In [ ]:
contact_frames = ['LF_FOOT', 'LH_FOOT', 'RF_FOOT', 'RH_FOOT']

Further, we have to explicitly set the contact type.  
Quadruped robots have point contacts, so we specify it as 

In [ ]:
contact_types = [robotoc.ContactType.PointContact for _ in contact_frames]

We also specify the time-step parameter (or weight parameters) of the Baumgarte's stabilization method.  
This is to take the accurate rigid contacts into account.  
Please see detail, e.g., in [this paper](https://www.researchgate.net/publication/234610391_A_Parametric_Study_on_the_Baumgarte_Stabilization_Method_for_Forward_Dynamics_of_Constrained_Multibody_Systems).
The appropriate parameter depends on the time step of MPC formulation.  
So some experimentation is necessary.   
In my experience, 2~10 times of time step of MPC work well.
Here, we set it as 0.04.

In [ ]:
baumgarte_time_step = 0.04

Finally, we construct the robot.  
Since the quadruped robot has a floating base, we specify it in the constructor.

In [ ]:
path_to_urdf = 'urdf/anymal_b_simple_description/urdf/anymal.urdf'
anymal = robotoc.Robot(path_to_urdf, robotoc.BaseJointType.FloatingBase, 
                       contact_frames, contact_types, baumgarte_time_step)

We can see the detail of the robot model as 

In [ ]:
print(anymal)

We can also display the robot.  

In [ ]:
steps = 10
q = [anymal.generate_feasible_configuration() for i in range(steps)]
dt = 1.0

viewer = robotoc.utils.TrajectoryViewer(path_to_urdf=path_to_urdf,
                                        base_joint_type=robotoc.BaseJointType.FloatingBase, 
                                        viewer_type='meshcat')
viewer.display(dt, q)

## Humanoid robot `iCub`

Third, we try a humanoid robot example.  
Humanoid robots typically involve **two surface contacts**.  
In the constructor, we specify the contact frames and types of the contacts as

In [ ]:
contact_frames = ['l_sole', 'r_sole']
contact_types = [robotoc.ContactType.SurfaceContact for i in contact_frames]

baumgarte_time_step = 0.05

path_to_urdf = 'urdf/icub_description/urdf/icub.urdf'
icub = robotoc.Robot(path_to_urdf, robotoc.BaseJointType.FloatingBase, 
                     contact_frames, contact_types, baumgarte_time_step)

We can see the detail of the robot model as 

In [ ]:
print(icub)

and display the robot

In [ ]:
steps = 10
q = [icub.generate_feasible_configuration() for i in range(steps)]
dt = 1.0

viewer = robotoc.utils.TrajectoryViewer(path_to_urdf=path_to_urdf,
                                        base_joint_type=robotoc.BaseJointType.FloatingBase, 
                                        viewer_type='meshcat')
viewer.display(dt, q)